In [ ]:
from scipy.optimize import minimize
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors

In [ ]:
data_p = 'C:\\Users\\lecorff\\Downloads\\SMCEM-Trans\\data\\'

### Synthetic data

In [ ]:
testdata = np.load(data_p + 'test_data_synthetic_d_8_10_particles_cv_0.npy')

In [ ]:
predlstm_unit_32_batch_32_cv_0_drop_10 = np.load(data_p + 'predlstm_unit_32_batch_32_cv_0_drop_0.10.npy')
predlstm_unit_32_batch_32_cv_0_drop_10_mean = np.mean(predlstm_unit_32_batch_32_cv_0_drop_10,axis=1)
predlstm_unit_32_batch_32_cv_0_drop_20 = np.load(data_p + 'predlstm_unit_32_batch_32_cv_0_drop_0.20.npy')
predlstm_unit_32_batch_32_cv_0_drop_20_mean = np.mean(predlstm_unit_32_batch_32_cv_0_drop_20,axis=1)
predlstm_unit_64_batch_32_cv_0_drop_20 = np.load(data_p + 'predlstm_unit_64_batch_32_cv_0_drop_0.20.npy')
predlstm_unit_64_batch_32_cv_0_drop_20_mean = np.mean(predlstm_unit_64_batch_32_cv_0_drop_20,axis=1)
predlstm_unit_64_batch_32_cv_0_drop_10 = np.load(data_p + 'predlstm_unit_64_batch_32_cv_0_drop_0.10.npy')
predlstm_unit_64_batch_32_cv_0_drop_10_mean = np.mean(predlstm_unit_64_batch_32_cv_0_drop_10,axis=1)

In [ ]:
pred_SMCT_d_16_30_particles_cv_0 = np.load(data_p + 'pred_SMCT_d_16_30_particles_cv_0.npy')
pred_SMCT_d_8_10_particles_cv_0 = np.load(data_p + 'pred_SMCT_d_8_10_particles_cv_0.npy')
pred_SMCT_d_8_10_particles_cv_0_mixt = np.load(data_p + 'pred_SMCT_d_8_10_particles_cv_0_mixt.npy')
pred_SMCT_d_16_30_particles_cv_0.shape

In [ ]:
predlstm_unit_32_batch_32_cv_0_drop_10.shape

In [ ]:
true_mean = 0.9*testdata
pred_SMCT_d_8_10_particles_cv_0_mean = np.mean(pred_SMCT_d_8_10_particles_cv_0,axis=1)
pred_SMCT_d_16_30_particles_cv_0_mean = np.mean(pred_SMCT_d_16_30_particles_cv_0,axis=1)
predlstm_unit_32_batch_32_cv_0_drop_10_mean = np.mean(predlstm_unit_32_batch_32_cv_0_drop_10,axis=1)
predlstm_unit_32_batch_32_cv_0_drop_10_std = np.std(predlstm_unit_32_batch_32_cv_0_drop_10,axis=1)
predlstm_unit_32_batch_32_cv_0_drop_20_mean = np.mean(predlstm_unit_32_batch_32_cv_0_drop_20,axis=1)
predlstm_unit_32_batch_32_cv_0_drop_20_std = np.std(predlstm_unit_32_batch_32_cv_0_drop_20,axis=1)
predlstm_unit_64_batch_32_cv_0_drop_20_mean = np.mean(predlstm_unit_64_batch_32_cv_0_drop_20,axis=1)
predlstm_unit_64_batch_32_cv_0_drop_20_std = np.std(predlstm_unit_64_batch_32_cv_0_drop_20,axis=1)
shift_x = 0.2
Nsim = 1000
xplot  = np.linspace(0,22,23)
xplotT = np.linspace(0,22,23) - shift_x
plt.figure(figsize=(15,10))
lst_pred = np.zeros((2,4,testdata.shape[2]-1))
lst_pred_2 = np.zeros((2,4,testdata.shape[2]-1))
for j in range(1):
    plt.subplot(1,1,j+1)
    idx_test = np.random.randint(0,testdata.shape[0]-1)
    plt.plot(true_mean[idx_test,0,0:-1,0], color = 'darkcyan',label='True mean',linewidth = 3)
    for tsp in range(testdata.shape[2]-1):
        lower_b = true_mean[idx_test,0,tsp,0]-1.96*np.sqrt(0.5)
        upper_b = true_mean[idx_test,0,tsp,0]+1.96*np.sqrt(0.5)
        Gauss = np.random.normal(0,1,Nsim)
        Gauss_2 = np.random.normal(0,1,Nsim)
        idx = np.sum(np.random.multinomial(1, np.ones(10)/10, Nsim)*np.arange(10),axis=1)
        idx_2 = np.sum(np.random.multinomial(1, np.ones(30)/30, Nsim)*np.arange(30),axis=1)
        lst_samp = []
        lst_samp_2 = []
        for i in range(Nsim):
            samp = pred_SMCT_d_8_10_particles_cv_0[idx_test,idx[i],tsp+1,0] + np.sqrt(0.54)*Gauss[i]
            samp_2 = pred_SMCT_d_16_30_particles_cv_0[idx_test,idx_2[i],tsp+1,0] + np.sqrt(0.49)*Gauss[i]
            lst_samp = np.append(lst_samp,samp)
            lst_samp_2 = np.append(lst_samp_2,samp_2)
            if samp>lower_b:
                if samp<upper_b:
                    lst_pred[j,0,tsp] = lst_pred[j,0,tsp] + 1
            if samp_2>lower_b:
                if samp_2<upper_b:
                    lst_pred_2[j,0,tsp] = lst_pred_2[j,0,tsp] + 1
        mean_samp = np.mean(lst_samp)
        std_samp = np.std(lst_samp)
        yy = np.linspace(lower_b,upper_b,10)
        zz_transf = np.linspace(mean_samp-1.96*std_samp,mean_samp+1.96*std_samp,10)
        zz = np.linspace(predlstm_unit_32_batch_32_cv_0_drop_10_mean[idx_test,tsp+1,0]-1.96*predlstm_unit_32_batch_32_cv_0_drop_10_std[idx_test,tsp+1,0],predlstm_unit_32_batch_32_cv_0_drop_10_mean[idx_test,tsp+1,0]+1.96*predlstm_unit_32_batch_32_cv_0_drop_10_std[idx_test,tsp+1,0],10)
        zz2 = np.linspace(predlstm_unit_32_batch_32_cv_0_drop_20_mean[idx_test,tsp+1,0]-1.96*predlstm_unit_32_batch_32_cv_0_drop_20_std[idx_test,tsp+1,0],predlstm_unit_32_batch_32_cv_0_drop_20_mean[idx_test,tsp+1,0]+1.96*predlstm_unit_32_batch_32_cv_0_drop_20_std[idx_test,tsp+1,0],10)
        zz3 = np.linspace(predlstm_unit_64_batch_32_cv_0_drop_20_mean[idx_test,tsp+1,0]-1.96*predlstm_unit_64_batch_32_cv_0_drop_20_std[idx_test,tsp+1,0],predlstm_unit_64_batch_32_cv_0_drop_20_mean[idx_test,tsp+1,0]+1.96*predlstm_unit_64_batch_32_cv_0_drop_20_std[idx_test,tsp+1,0],10)
        xx = tsp*np.ones(10)
        xx_samp = tsp*np.ones(Nsim)
        if tsp == 0:
            plt.plot(xx,yy,color = 'darkcyan', label = 'True 95% confidence interval', linewidth = 3)
            #plt.plot(xx_samp-shift_x,lst_samp,color = 'salmon', label = 'Samples from estimated distribution  - SMC-Transformer (d = 8, M = 10)', marker = 'o', markersize = 3, alpha = 0.8,linewidth = 0)
            plt.plot(xx_samp-shift_x,lst_samp_2,color = 'firebrick', label = 'Samples from estimated distribution  - SMC-Transformer (d = 16, M = 30)', marker = 'o', markersize = 4, alpha = 0.8,linewidth = 0)
            #plt.plot(xx+shift_x,zz,color = 'limegreen', label = 'Samples - LSTM (d = 32, drop = 0.1)', marker = 'o', markersize = 3, alpha = 0.8,linewidth = 0)
            plt.plot(xx+shift_x,zz2,color = 'seagreen', label = 'Samples - LSTM (d = 32, drop = 0.2)', marker = 'o', markersize = 4, alpha = 0.8,linewidth = 0)
            plt.plot(xx+2*shift_x,zz3,color = 'darkgreen', label = 'Samples - LSTM (d = 64, drop = 0.2)', marker = 'o', markersize = 4, alpha = 0.8,linewidth = 0)
        else:
            plt.plot(xx,yy,color = 'darkcyan',linewidth = 3)  
            plt.plot(xx_samp-shift_x,lst_samp_2,color = 'firebrick', marker = 'o', markersize = 4, alpha = 0.8,linewidth = 0)
            #plt.plot(xx_samp-shift_x,lst_samp,color = 'salmon', marker = 'o', markersize = 3, alpha = 0.8,linewidth = 0)
            #plt.plot(xx+shift_x,zz,color = 'limegreen', marker = 'o', markersize = 2, alpha = 0.8,linewidth = 0) 
            plt.plot(xx+shift_x,zz2,color = 'seagreen', marker = 'o', markersize = 4, alpha = 0.8,linewidth = 0)
            plt.plot(xx+2*shift_x,zz3,color = 'darkgreen', marker = 'o', markersize = 4, alpha = 0.8,linewidth = 0)
    plt.ylabel('Signal values', fontsize=16)
    plt.xlabel('Time steps', fontsize=16)
    plt.grid('on')
    plt.legend(markerscale=3, fontsize=14,frameon=False)
lst_pred = lst_pred/Nsim

In [ ]:
true_mean = 0.9*testdata
pred_SMCT_d_8_10_particles_cv_0_mean = np.mean(pred_SMCT_d_8_10_particles_cv_0,axis=1)
predlstm_unit_32_batch_32_cv_0_drop_10_mean = np.mean(predlstm_unit_32_batch_32_cv_0_drop_10,axis=1)
predlstm_unit_32_batch_32_cv_0_drop_10_std = np.std(predlstm_unit_32_batch_32_cv_0_drop_10,axis=1)
predlstm_unit_32_batch_32_cv_0_drop_20_mean = np.mean(predlstm_unit_32_batch_32_cv_0_drop_20,axis=1)
predlstm_unit_32_batch_32_cv_0_drop_20_std = np.std(predlstm_unit_32_batch_32_cv_0_drop_20,axis=1)
predlstm_unit_64_batch_32_cv_0_drop_20_mean = np.mean(predlstm_unit_64_batch_32_cv_0_drop_20,axis=1)
predlstm_unit_64_batch_32_cv_0_drop_20_std = np.std(predlstm_unit_64_batch_32_cv_0_drop_20,axis=1)
shift_x = 0.1
Nsim = 500
xplot  = np.linspace(0,22,23)
xplotT = np.linspace(0,22,23) + shift_x
plt.figure(figsize=(14,10))
lst_pred = np.zeros((testdata.shape[0],4,testdata.shape[2]-1))
lst_pred_2 = np.zeros((testdata.shape[0],4,testdata.shape[2]-1))
mse_true = []
mse_10 = []
mse_30 = []
mse_64_01 = []
mse_64_02 = []
mse_32_01 = []
mse_32_02 = []
for idx_test in range(testdata.shape[0]):
    for tsp in range(testdata.shape[2]-1):
        lower_b = true_mean[idx_test,0,tsp,0]-1.96*np.sqrt(0.5)
        upper_b = true_mean[idx_test,0,tsp,0]+1.96*np.sqrt(0.5)
        Gauss = np.random.normal(0,1,Nsim)
        Gauss_2 = np.random.normal(0,1,Nsim)
        idx_1 = np.sum(np.random.multinomial(1, np.ones(10)/10, Nsim)*np.arange(10),axis=1)
        idx_2 = np.sum(np.random.multinomial(1, np.ones(30)/30, Nsim)*np.arange(30),axis=1)
        lst_samp = []
        lst_samp_2 = []
        lst_samp_true = []
        for i in range(Nsim):
            samp = pred_SMCT_d_8_10_particles_cv_0[idx_test,idx[i],tsp+1,0] + np.sqrt(0.54)*Gauss[i]
            lst_samp = np.append(lst_samp,samp)
            samp_2 = pred_SMCT_d_16_30_particles_cv_0[idx_test,idx[i],tsp+1,0] + np.sqrt(0.50)*Gauss_2[i]
            lst_samp_2 = np.append(lst_samp_2,samp_2)
            samp_true = true_mean[idx_test,0,tsp,0] + np.sqrt(0.5)*Gauss[i]
            lst_samp_true = np.append(lst_samp_true,samp_true)
            if samp>lower_b:
                if samp<upper_b:
                    lst_pred[idx_test,0,tsp] = lst_pred[idx_test,0,tsp] + 1
            if samp_2>lower_b:
                if samp_2<upper_b:
                    lst_pred_2[idx_test,0,tsp] = lst_pred_2[idx_test,0,tsp] + 1
        
        mse_true = np.append(mse_true,np.mean((lst_samp_true-true_mean[idx_test,0,tsp,0])**2))
        mse_10 = np.append(mse_10,np.mean((lst_samp-true_mean[idx_test,0,tsp,0])**2))
        mse_30 = np.append(mse_30,np.mean((lst_samp_2-true_mean[idx_test,0,tsp,0])**2))
        mse_64_01 = np.append(mse_64_01,np.mean((predlstm_unit_64_batch_32_cv_0_drop_10[idx_test,:,tsp+1,0]-true_mean[idx_test,0,tsp,0])**2))
        mse_64_02 = np.append(mse_64_02,np.mean((predlstm_unit_64_batch_32_cv_0_drop_20[idx_test,:,tsp+1,0]-true_mean[idx_test,0,tsp,0])**2))
        mse_32_01 = np.append(mse_32_01,np.mean((predlstm_unit_32_batch_32_cv_0_drop_10[idx_test,:,tsp+1,0]-true_mean[idx_test,0,tsp,0])**2))
        mse_32_02 = np.append(mse_32_02,np.mean((predlstm_unit_32_batch_32_cv_0_drop_20[idx_test,:,tsp+1,0]-true_mean[idx_test,0,tsp,0])**2))        
lst_pred = lst_pred/Nsim
lst_pred_2 = lst_pred_2/Nsim

In [ ]:
print('Mean true model: %.3f  '%np.mean(mse_true) + '  Std true model: %.3f'%np.std(mse_true))
print('Mean 10 particles: %.3f  '%np.mean(mse_10) + '  Std 10 particles: %.3f'%np.std(mse_10))
print('Mean 30 particles: %.3f  '%np.mean(mse_30) + '  Std 30 particles: %.3f'%np.std(mse_30))
print('Mean d = 32 drop = 0.1: %.3f  '%np.mean(mse_32_01) + '  Std 30 particles: %.3f'%np.std(mse_32_01))
print('Mean d = 32 drop = 0.2: %.3f  '%np.mean(mse_32_02) + '  Std 30 particles: %.3f'%np.std(mse_32_02))
print('Mean d = 64 drop = 0.1: %.3f  '%np.mean(mse_64_01) + '  Std 30 particles: %.3f'%np.std(mse_64_01))
print('Mean d = 64 drop = 0.2: %.3f  '%np.mean(mse_64_02) + '  Std 30 particles: %.3f'%np.std(mse_64_02))

In [ ]:
mean_lst_pred = np.mean(lst_pred,axis=0)
mean_lst_pred_2 = np.mean(lst_pred_2,axis=0)
np.reshape(lst_pred[:,0,:],100*23).shape
data = {'SMC-Transformer (d = 8, M = 10)': np.reshape(lst_pred[:,0,:],100*23), 'SMC-Transformer (d = 16, M = 30)': np.reshape(lst_pred_2[:,0,:],100*23)}
df = pd. DataFrame(data, columns = ['SMC-Transformer (d = 8, M = 10)','SMC-Transformer (d = 16, M = 30)'])

In [ ]:
sns.boxplot(df['SMC-Transformer (d = 8, M = 10)'])

In [ ]:
sns.boxplot(df['SMC-Transformer (d = 16, M = 30)'])

In [ ]:
plt.figure(figsize=(14,10))
for j in range(lst_pred.shape[0]-10):
    if np.mean(lst_pred[j,0,:])>0.85:
        if j ==0:
            plt.plot(np.arange(0,23),lst_pred[j,0,:],color = 'limegreen', marker = 'o', markersize = 4, alpha = 0.8,linewidth = 0,label = 'Frequency of samples in the true confidence interval - SMC-Transformer (d = 8, M = 10)')
            plt.plot(np.arange(0,23) + 3*shift_x,lst_pred_2[j,0,:],color = 'darkcyan', marker = 'o', markersize = 4, alpha = 0.8,linewidth = 0,label = 'Frequency of samples in the true confidence interval - SMC-Transformer (d = 16, M = 30)')
        else:
            plt.plot(np.arange(0,23),lst_pred[j,0,:],color = 'limegreen', marker = 'o', markersize = 4, alpha = 0.8,linewidth = 0)  
            plt.plot(np.arange(0,23) + 3*shift_x,lst_pred_2[j,0,:],color = 'darkcyan', marker = 'o', markersize = 4, alpha = 0.8,linewidth = 0) 
plt.plot(mean_lst_pred[0,:],color = 'limegreen', alpha = 0.8,linewidth = 2,label = 'Mean over the 100 test examples - SMC-Transformer (d = 8, M = 10)') 
plt.plot(np.arange(0,23) + 3*shift_x,mean_lst_pred_2[0,:],color = 'darkcyan', alpha = 0.8,linewidth = 2,label = 'Mean over the 100 test examples - SMC-Transformer (d = 16, M = 30)') 
plt.ylim([0.85,0.98])
plt.ylabel('Frequency of predicted samples in the true confidence intervals', fontsize=12)
plt.xlabel('Time steps', fontsize=12)
plt.grid('on')
plt.legend(markerscale=2, fontsize=12)

In [ ]:
from sklearn.neighbors import KernelDensity

In [ ]:
from scipy.stats.kde import gaussian_kde
from scipy.stats import norm

In [ ]:
plt.figure(figsize=(14,14))
for j in range(4):
    plt.subplot(2,2,j+1)
    idx_test = np.random.randint(0,testdata.shape[0]-1)
    timestep = np.random.randint(0,testdata.shape[2]-1)
    plt.title('Test sample %d - time step %d'%(idx_test,timestep))
    true_mean = 0.8*testdata[idx_test,0,timestep-1,0]
    X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
    kde_lstm_unit_32_batch_32_cv_0_drop_20 = gaussian_kde(predlstm_unit_32_batch_32_cv_0_drop_20[idx_test,:,timestep,0])
    xx = np.linspace(true_mean-1,true_mean+1,50)
    plt.plot(xx,norm.pdf(xx,true_mean,np.sqrt(0.5)), color = 'darkcyan',label='True probability density',linewidth = 3)
    
    #plt.plot(xx, kde_lstm_unit_32_batch_32_cv_0_drop_20(xx),color = 'limegreen', label = 'Samples - LSTM (d = 32, drop = 0.1)', marker = 'o', markersize = 2, alpha = 0.8,linewidth = 0)
    
    u = np.zeros(50)
    for j in range(30):
        u = u + norm.pdf(xx,pred_SMCT_d_16_30_particles_cv_0[idx_test,j,timestep,0],np.sqrt(.49))
    u = u/30
    plt.plot(xx,u, label='Predicted_pdf - SMC-Transformer (d = 16, M = 30)',linewidth = 4,alpha= 0.5, linestyle = "dashed")
    plt.ylabel('Probability density', fontsize=12)
    plt.grid('on')
    u = np.zeros(50)
    for j in range(10):
        u = u + norm.pdf(xx,pred_SMCT_d_8_10_particles_cv_0[idx_test,j,timestep,0],np.sqrt(0.54))
    u = u/10
    plt.plot(xx,u, label='Predicted_pdf - SMC-Transformer (d = 8, M = 10)',linewidth = 4,alpha= 0.5, linestyle = "dashed")
    plt.ylabel('Probability density', fontsize=12)
    plt.grid('on')
    plt.ylabel('Probability density', fontsize=12)
    plt.grid('on')
    plt.legend()

### Synrhetic data from a mixture distribution

In [ ]:
pred_SMCT_d_32_10_particles_cv_0_mixt = np.load(data_p + 'pred_SMCT_d_32_10_particles_cv_0_mixt.npy')
predlstm_unit_32_batch_32_cv_0_drop_10_mixt = np.load(data_p + 'predlstm_unit_32_batch_32_cv_0_drop_0.10_mixt.npy')
predlstm_unit_32_batch_32_cv_0_drop_20_mixt = np.load(data_p + 'predlstm_unit_32_batch_32_cv_0_drop_0.20_mixt.npy')
predlstm_unit_64_batch_32_cv_0_drop_10_mixt = np.load(data_p + 'predlstm_unit_64_batch_32_cv_0_drop_0.10_mixt.npy')
predlstm_unit_64_batch_32_cv_0_drop_20_mixt = np.load(data_p + 'predlstm_unit_64_batch_32_cv_0_drop_0.20_mixt.npy')

In [ ]:
test_data_synthetic_d_32_10_particles_cv_0_mixt = np.load(data_p + 'test_data_synthetic_unit_128_batch_32_cv_0_drop_0.20_mixt.npy')
test_data_synthetic_d_32_10_particles_cv_0_mixt.shape

In [ ]:
shift_x = 0.2
Nsim = 500
plt.figure(figsize=(15,10))
lst_pred = np.zeros((2,4,testdata.shape[2]-1))
for j in range(1):
    plt.subplot(1,1,j+1)
    idx_test = np.random.randint(0,testdata.shape[0]-1)
    plt.plot(test_data_synthetic_d_32_10_particles_cv_0_mixt[idx_test,:-1,0], 
             color = 'darkcyan',label='True observations',linewidth = 3)
    for tsp in range(testdata.shape[2]-1):
        #lower_b = true_mean[idx_test,tsp]-1.96*np.sqrt(0.5)
        #upper_b = true_mean[idx_test,tsp]+1.96*np.sqrt(0.5)
        Gauss = np.random.normal(0,1,Nsim)
        Gauss_2 = np.random.normal(0,1,Nsim)
        idx = np.sum(np.random.multinomial(1, np.ones(10)/10, Nsim)*np.arange(10),axis=1)
        #idx_2 = np.sum(np.random.multinomial(1, np.ones(30)/30, Nsim)*np.arange(30),axis=1)
        lst_samp = []
        lst_samp_true = []
        #lst_samp_2 = []
        #plt.plot(pred_SMCT_d_16_10_particles_cv_0_mixt[idx_test,idx[i],3::,0])
        for i in range(Nsim):
            samp = pred_SMCT_d_32_10_particles_cv_0_mixt[idx_test,idx[i],tsp+1,0] + np.sqrt(0.33)*Gauss[i]
            U = np.random.uniform()
            if U<0.7:
                samp_true = 0.9*test_data_synthetic_d_32_10_particles_cv_0_mixt[idx_test,tsp,0] + np.sqrt(0.3)*Gauss_2[i]
            else:
                samp_true = 0.9*0.6*test_data_synthetic_d_32_10_particles_cv_0_mixt[idx_test,tsp,0] + np.sqrt(0.3)*Gauss_2[i]
            lst_samp = np.append(lst_samp,samp)
            lst_samp_true = np.append(lst_samp_true,samp_true)
            #lst_samp_2 = np.append(lst_samp_2,samp_2)
            #if samp>lower_b:
            #    if samp<upper_b:
            #        lst_pred[j,0,tsp] = lst_pred[j,0,tsp] + 1
            #if samp_2>lower_b:
            #    if samp_2<upper_b:
            #        lst_pred_2[j,0,tsp] = lst_pred_2[j,0,tsp] + 1
        mean_samp = np.mean(lst_samp)
        std_samp = np.std(lst_samp)
        #yy = np.linspace(lower_b,upper_b,10)
        #zz_transf = np.linspace(mean_samp-1.96*std_samp,mean_samp+1.96*std_samp,10)
        #zz = np.linspace(predlstm_unit_32_batch_32_cv_0_drop_10_mean[idx_test,tsp+1,0]-1.96*predlstm_unit_32_batch_32_cv_0_drop_10_std[idx_test,tsp+1,0],predlstm_unit_32_batch_32_cv_0_drop_10_mean[idx_test,tsp+1,0]+1.96*predlstm_unit_32_batch_32_cv_0_drop_10_std[idx_test,tsp+1,0],10)
        #zz2 = np.linspace(predlstm_unit_32_batch_32_cv_0_drop_20_mean[idx_test,tsp+1,0]-1.96*predlstm_unit_32_batch_32_cv_0_drop_20_std[idx_test,tsp+1,0],predlstm_unit_32_batch_32_cv_0_drop_20_mean[idx_test,tsp+1,0]+1.96*predlstm_unit_32_batch_32_cv_0_drop_20_std[idx_test,tsp+1,0],10)
        #zz3 = np.linspace(predlstm_unit_64_batch_32_cv_0_drop_20_mean[idx_test,tsp+1,0]-1.96*predlstm_unit_64_batch_32_cv_0_drop_20_std[idx_test,tsp+1,0],predlstm_unit_64_batch_32_cv_0_drop_20_mean[idx_test,tsp+1,0]+1.96*predlstm_unit_64_batch_32_cv_0_drop_20_std[idx_test,tsp+1,0],10)
        xx = tsp*np.ones(10)
        xx_samp = tsp*np.ones(Nsim)
        if tsp == 0:
            plt.plot(xx_samp,lst_samp_true,color = 'darkcyan', label = 'Samples from the true distribution', marker = 'o', markersize = 3, linewidth = 0)
            plt.plot(xx_samp-shift_x,lst_samp,color = 'salmon', label = 'Samples from estimated distribution  - SMC-Transformer (d = 32, M = 10)', marker = 'o', markersize = 3, alpha = 0.8,linewidth = 0)
            #plt.plot(xx_samp-shift_x,lst_samp,color = 'firebrick', label = 'Samples from estimated distribution  - SMC-Transformer (d = 16, M = 30)', marker = 'o', markersize = 4, alpha = 0.8,linewidth = 0)
            plt.plot(xx_samp+shift_x,predlstm_unit_32_batch_32_cv_0_drop_10_mixt[idx_test,:,tsp,0],color = 'limegreen', label = 'Samples - LSTM (d = 32, drop = 0.1)', marker = 'o', markersize = 3, alpha = 0.8,linewidth = 0)
            plt.plot(xx_samp+2*shift_x,predlstm_unit_64_batch_32_cv_0_drop_10_mixt[idx_test,:,tsp,0],color = 'seagreen', label = 'Samples - LSTM (d = 64, drop = 0.1)', marker = 'o', markersize = 4, alpha = 0.8,linewidth = 0)
            plt.plot(xx_samp+3*shift_x,predlstm_unit_64_batch_32_cv_0_drop_20_mixt[idx_test,:,tsp,0],color = 'darkgreen', label = 'Samples - LSTM (d = 64, drop = 0.2)', marker = 'o', markersize = 4, alpha = 0.8,linewidth = 0)
        else:
            plt.plot(xx_samp,lst_samp_true,color = 'darkcyan', marker = 'o', markersize = 3, linewidth = 0)  
            #plt.plot(xx_samp-shift_x,lst_samp,color = 'firebrick', marker = 'o', markersize = 4, alpha = 0.8,linewidth = 0)
            plt.plot(xx_samp-shift_x,lst_samp,color = 'salmon', marker = 'o', markersize = 3, alpha = 0.8,linewidth = 0)
            plt.plot(xx_samp+shift_x,predlstm_unit_32_batch_32_cv_0_drop_10_mixt[idx_test,:,tsp,0],color = 'limegreen', marker = 'o', markersize = 3, alpha = 0.8,linewidth = 0)
            plt.plot(xx_samp+2*shift_x,predlstm_unit_64_batch_32_cv_0_drop_10_mixt[idx_test,:,tsp,0],color = 'seagreen', marker = 'o', markersize = 4, alpha = 0.8,linewidth = 0)
            plt.plot(xx_samp+3*shift_x,predlstm_unit_64_batch_32_cv_0_drop_20_mixt[idx_test,:,tsp,0],color = 'darkgreen', marker = 'o', markersize = 4, alpha = 0.8,linewidth = 0)
    plt.ylabel('Signal values', fontsize=16)
    plt.xlabel('Time steps', fontsize=16)
    plt.grid('on')
    plt.legend(markerscale=3, fontsize=14,frameon=False)
lst_pred = lst_pred/Nsim

In [ ]:
true_mean = 0.9*test_data_synthetic_d_32_10_particles_cv_0_mixt
true_mean_2 = 0.54*test_data_synthetic_d_32_10_particles_cv_0_mixt
pred_SMCT_d_32_10_particles_cv_0_mean = np.mean(pred_SMCT_d_32_10_particles_cv_0_mixt,axis=1)
predlstm_unit_32_batch_32_cv_0_drop_10_mean = np.mean(predlstm_unit_32_batch_32_cv_0_drop_10,axis=1)
predlstm_unit_32_batch_32_cv_0_drop_10_std = np.std(predlstm_unit_32_batch_32_cv_0_drop_10,axis=1)
predlstm_unit_32_batch_32_cv_0_drop_20_mean = np.mean(predlstm_unit_32_batch_32_cv_0_drop_20,axis=1)
predlstm_unit_32_batch_32_cv_0_drop_20_std = np.std(predlstm_unit_32_batch_32_cv_0_drop_20,axis=1)
predlstm_unit_64_batch_32_cv_0_drop_20_mean = np.mean(predlstm_unit_64_batch_32_cv_0_drop_20,axis=1)
predlstm_unit_64_batch_32_cv_0_drop_20_std = np.std(predlstm_unit_64_batch_32_cv_0_drop_20,axis=1)
shift_x = 0.1
Nsim = 500
xplot  = np.linspace(0,22,23)
xplotT = np.linspace(0,22,23) + shift_x
plt.figure(figsize=(14,10))
lst_pred = np.zeros((testdata.shape[0],4,testdata.shape[2]-1))
lst_pred_2 = np.zeros((testdata.shape[0],4,testdata.shape[2]-1))
mse_true = []
mse_10 = []
mse_30 = []
mse_64_01 = []
mse_64_02 = []
mse_32_01 = []
mse_32_02 = []
for idx_test in range(testdata.shape[0]):
    for tsp in range(testdata.shape[2]-1):
        #lower_b = true_mean[idx_test,0,tsp,0]-1.96*np.sqrt(0.5)
        #upper_b = true_mean[idx_test,0,tsp,0]+1.96*np.sqrt(0.5)
        Gauss = np.random.normal(0,1,Nsim)
        #Gauss_2 = np.random.normal(0,1,Nsim)
        idx_1 = np.sum(np.random.multinomial(1, np.ones(10)/10, Nsim)*np.arange(10),axis=1)
        #idx_2 = np.sum(np.random.multinomial(1, np.ones(30)/30, Nsim)*np.arange(30),axis=1)
        lst_samp = []
        #lst_samp_2 = []
        lst_samp_true = []
        for i in range(Nsim):
            samp = pred_SMCT_d_32_10_particles_cv_0_mixt[idx_test,idx_1[i],tsp+1,0] + np.sqrt(0.33)*Gauss[i]
            lst_samp = np.append(lst_samp,samp)
            #samp_2 = pred_SMCT_d_16_30_particles_cv_0[idx_test,idx[i],tsp+1,0] + np.sqrt(0.33)*Gauss_2[i]
            #lst_samp_2 = np.append(lst_samp_2,samp_2)
            U = np.random.uniform()
            if U<0.7:
                samp_true = 0.9*test_data_synthetic_d_32_10_particles_cv_0_mixt[idx_test,tsp,0] + np.sqrt(0.3)*Gauss_2[i]
            else:
                samp_true = 0.9*0.6*test_data_synthetic_d_32_10_particles_cv_0_mixt[idx_test,tsp,0] + np.sqrt(0.3)*Gauss_2[i]
            lst_samp = np.append(lst_samp,samp)
            lst_samp_true = np.append(lst_samp_true,samp_true)
            if samp>lower_b:
                if samp<upper_b:
                    lst_pred[idx_test,0,tsp] = lst_pred[idx_test,0,tsp] + 1
            #if samp_2>lower_b:
            #    if samp_2<upper_b:
            #        lst_pred_2[idx_test,0,tsp] = lst_pred_2[idx_test,0,tsp] + 1
        mse_true = np.append(mse_true,0.7*np.mean((lst_samp_true-true_mean[idx_test,tsp,0])**2)
                             + 0.3*np.mean((lst_samp_true-true_mean_2[idx_test,tsp,0])**2))
        mse_10 = np.append(mse_10,0.7*np.mean((lst_samp-true_mean[idx_test,tsp,0])**2) + 0.3*np.mean((lst_samp-true_mean_2[idx_test,tsp,0])**2))
        #mse_30 = np.append(mse_30,np.mean((lst_samp_2-true_mean[idx_test,0,tsp,0])**2))
        mse_64_01 = np.append(mse_64_01,0.7*np.mean((predlstm_unit_64_batch_32_cv_0_drop_10[idx_test,:,tsp+1,0]-true_mean[idx_test,tsp,0])**2) + 0.3*np.mean((predlstm_unit_64_batch_32_cv_0_drop_10[idx_test,:,tsp+1,0]-true_mean_2[idx_test,tsp,0])**2))
        mse_64_02 = np.append(mse_64_02,0.7*np.mean((predlstm_unit_64_batch_32_cv_0_drop_20[idx_test,:,tsp+1,0]-true_mean[idx_test,tsp,0])**2) + 0.3*np.mean((predlstm_unit_64_batch_32_cv_0_drop_20[idx_test,:,tsp+1,0]-true_mean_2[idx_test,tsp,0])**2))
        mse_32_01 = np.append(mse_32_01,0.7*np.mean((predlstm_unit_32_batch_32_cv_0_drop_10[idx_test,:,tsp+1,0]-true_mean[idx_test,tsp,0])**2) + 0.3*np.mean((predlstm_unit_32_batch_32_cv_0_drop_10[idx_test,:,tsp+1,0]-true_mean_2[idx_test,tsp,0])**2))
        mse_32_02 = np.append(mse_32_02,0.7*np.mean((predlstm_unit_32_batch_32_cv_0_drop_20[idx_test,:,tsp+1,0]-true_mean[idx_test,tsp,0])**2) + 0.3*np.mean((predlstm_unit_32_batch_32_cv_0_drop_20[idx_test,:,tsp+1,0]-true_mean_2[idx_test,tsp,0])**2))       
lst_pred = lst_pred/Nsim
lst_pred_2 = lst_pred_2/Nsim

In [ ]:
print('Mean true: %.3f  '%np.mean(mse_true) + '  Std true: %.3f'%np.std(mse_true))
print('Mean 10 particles: %.3f  '%np.mean(mse_10) + '  Std 10 particles: %.3f'%np.std(mse_10))
#print('Mean 30 particles: %.3f'%np.mean(mse_30) + '  Std 30 particles: %.3f'%np.std(mse_30))
print('Mean d = 32 drop = 0.1: %.3f  '%np.mean(mse_32_01) + '  Std 30 particles: %.3f'%np.std(mse_32_01))
print('Mean d = 32 drop = 0.2: %.3f  '%np.mean(mse_32_02) + '  Std 30 particles: %.3f'%np.std(mse_32_02))
print('Mean d = 64 drop = 0.1: %.3f  '%np.mean(mse_64_01) + '  Std 30 particles: %.3f'%np.std(mse_64_01))
print('Mean d = 64 drop = 0.2: %.3f  '%np.mean(mse_64_02) + '  Std 30 particles: %.3f'%np.std(mse_64_02))

In [ ]:
plt.figure(figsize=(14,14))
for j in range(4):
    plt.subplot(2,2,j+1)
    idx_test = np.random.randint(0,testdata.shape[0]-1)
    timestep = np.random.randint(0,testdata.shape[2]-1)
    plt.title('Test sample %d - time step %d'%(idx_test,timestep))
    true_mean_1 = 0.9*test_data_synthetic_d_32_10_particles_cv_0_mixt[idx_test,timestep-1,0]
    true_mean_2 = 0.2*test_data_synthetic_d_32_10_particles_cv_0_mixt[idx_test,timestep-1,0]
    true_mean = 0.7*true_mean_1 + 0.3*true_mean_2
    xx = np.linspace(true_mean-1,true_mean+1,50)
    plt.plot(xx,0.7*norm.pdf(xx,true_mean_1,np.sqrt(0.3))+0.3*norm.pdf(xx,true_mean_2,np.sqrt(0.3)), color = 'darkcyan',label='True probability density',linewidth = 3)
    
    #plt.plot(xx, kde_lstm_unit_32_batch_32_cv_0_drop_20(xx),color = 'limegreen', label = 'Samples - LSTM (d = 32, drop = 0.1)', marker = 'o', markersize = 2, alpha = 0.8,linewidth = 0)
    
    u = np.zeros(50)
    for j in range(10):
        u = u + norm.pdf(xx,pred_SMCT_d_32_10_particles_cv_0_mixt[idx_test,j,timestep,0],np.sqrt(.33))
    u = u/10
    
    plt.plot(xx,u, label='Predicted_pdf - SMC-Transformer (d = 32, M = 10)',linewidth = 4,alpha= 0.5, linestyle = "dashed")
    plt.ylabel('Probability density', fontsize=12)
    plt.grid('on')
    plt.legend()

### Covid data

In [ ]:
covid_data_path  = data_p + 'time_series_covid19_deaths_US.csv'
df = pd.read_csv(covid_data_path)
df.shape

In [ ]:
df = pd.read_csv(covid_data_path)
df = df[df['5/6/20']>10]
df = df[df['5/6/20']<1000]

In [ ]:
scale_fact = df['Population']/1e2

In [ ]:
Province_State = df['Province_State'].values
Admin2 = df['Admin2'].values

In [ ]:
df.drop(df.columns[[0,1,2,3,4,5,6,7,8,9,10,11]], axis=1, inplace=True)
df.shape

In [ ]:
df.values
df_scaled = np.zeros((df.shape[0],df.shape[1]))
for j in range(df.shape[1]):
    df_scaled[:,j] = df.values[:,j]/scale_fact

In [ ]:
df_scaled

In [ ]:
df.head()

In [ ]:
df.shape[0]

In [ ]:
df[j][60::]

In [ ]:
cov_data = np.zeros((df.shape[0],46,1))
for j in range(df.shape[0]-1):
    cov_data[j,:,0] = df.values[j][60::]
covdata_data_path = data_p + 'cov_data.npy'
np.save(covdata_data_path,cov_data)

In [ ]:
covdata = np.load(data_p + 'testcov.npy')
covpred_part = np.load(data_p + 'predcov_part.npy')
covpred_6 = np.load(data_p + 'predcov_6.npy')
covpred_12 = np.load(data_p + 'predcov_12.npy')
covpred_36 = np.load(data_p + 'predcov_36.npy')
covpred_128 = np.load(data_p + 'predcov_128.npy')
covpredlstm_6 = np.load(data_p + 'predlstm_6.npy')
covpredlstm_12 = np.load(data_p + 'predlstm_12.npy')
covpredlstm_36 = np.load(data_p + 'predlstm_36.npy')
covpredlstm_128 = np.load(data_p + 'predlstm_128.npy')

In [ ]:
covpred_part.shape

### Models without noise

In [ ]:
testdata = np.load(data_p + 'test_data_synthetic_d_12_30_particles_cov_idx_0.npy')
preddata_30 = np.load(data_p + 'pred_SMCT_d_12_30_particles_cov_idx_0_var_21.73.npy')
testdata.shape

In [ ]:
preddata_30 = np.load(data_p + 'pred_SMCT_d_12_30_particles_cov_idx_0_var_21.73.npy')

In [ ]:
part_mean = np.mean(preddata_30,axis=1)
shift_x = 0
Nsim = 500
xplot = np.linspace(0,22,23) + shift_x
plt.figure(figsize=(18,18))
for j in range(6):
    plt.subplot(3,2,j+1)
    idx_test = np.random.randint(0,testdata.shape[0]-1)
    plt.plot(true_mean[idx_test,0,0:-1,0], 'ro',label='True mean',linewidth = 1)
    plt.plot(xplot,part_mean[idx_test,1::,0], 'ko',label='Estimated mean',linewidth = 1)
    plt.plot(xplot,preddata_30[idx_test,:,1::,0].T, 'k.',alpha = 0.5,linewidth = 1)
    for tsp in range(testdata.shape[2]-1):
        Gauss = np.random.normal(0,1,Nsim)
        idx = np.sum(np.random.multinomial(1, np.ones(30)/30, Nsim)*np.arange(30),axis=1)
        lst_samp = []
        for i in range(Nsim):
            samp = preddata_30[idx_test,idx[i],tsp+1,0] + np.sqrt(0.55)*Gauss[idx[i]]
            lst_samp = np.append(lst_samp,samp)
        mean_samp = np.mean(lst_samp)
        std_samp = np.std(lst_samp)
        yy = np.linspace(true_mean[idx_test,0,tsp,0]-1.96*np.sqrt(0.5),true_mean[idx_test,0,tsp,0]+1.96*np.sqrt(0.5),10)
        zz = np.linspace(mean_samp-1.96*std_samp,mean_samp+1.96*std_samp,10)
        xx = tsp*np.ones(10)
        if tsp == 0:
            plt.plot(xx,yy,'r', label = 'True 95% confidence interval')
            plt.plot(xx+shift_x,zz,'k', label = 'Predicted 95% confidence interval')
        else:
            plt.plot(xx,yy,'r')  
            plt.plot(xx+shift_x,zz,'k') 
    plt.ylabel('Signal values', fontsize=12)
    plt.xlabel('Time steps', fontsize=12)
    plt.grid('on')
    plt.legend()

In [ ]:
plt.figure(figsize=(16,16))
for j in range(6):
    plt.subplot(3,2,j+1)
    idx_test = np.random.randint(0,covdata.shape[0])
    plt.plot(covdata[idx_test,0,:,0], 'r',linewidth = 2, label='True data - ' + Admin2[idx_test] + ' ; ' + Province_State[idx_test])
    #mse = np.sum((covdata[idx_test,0,:,0]-covpred_6[idx_test,:,:,0])*(covdata[idx_test,0,:,0]-covpred_6[idx_test,:,:,0]))/covdata[idx_test,0,:,0].shape[0]
    #plt.plot(np.floor(covpred_6[idx_test,:,:,0].T),'m.',linewidth = 1,alpha= 0.6 ,linestyle = "dashed", label='Transf. d = 6 - MSE = %f'%mse)
    mse = np.sum((covdata[idx_test,0,:,0]-covpred_12[idx_test,:,:,0])*(covdata[idx_test,0,:,0]-covpred_12[idx_test,:,:,0]))/covdata[idx_test,0,:,0].shape[0]
    plt.plot(np.floor(covpred_12[idx_test,:,:,0].T),color = 'darkcyan',marker = 'o',linewidth = 1,alpha= 0.6,linestyle = "dashed", label='Transf. d = 12 - MSE = %.2f'%mse)
    mse = np.sum((covdata[idx_test,0,:,0]-covpred_36[idx_test,:,:,0])*(covdata[idx_test,0,:,0]-covpred_36[idx_test,:,:,0]))/covdata[idx_test,0,:,0].shape[0]
    plt.plot(np.floor(covpred_36[idx_test,:,:,0].T),color = 'limegreen',marker = 'o',linewidth = 1,alpha= 0.6,linestyle = "dashed", label='Transf. d = 36 - MSE = %.2f'%mse) 
    #mse = np.sum((covdata[idx_test,0,:,0]-covpred_128[idx_test,:,:,0])*(covdata[idx_test,0,:,0]-covpred_128[idx_test,:,:,0]))/covdata[idx_test,0,:,0].shape[0]
    #plt.plot(np.floor(covpred_128[idx_test,:,:,0].T),'mo',linewidth = 1,alpha= 0.6,linestyle = "dashed", label='Transf. d = 128 - MSE = %.2f'%mse)
    #mse = np.sum((covdata[idx_test,0,:,0]-covpredlstm_6[idx_test,:,0])*(covdata[idx_test,0,:,0]-covpredlstm_6[idx_test,:,0]))/covdata[idx_test,0,:,0].shape[0]
    #plt.plot(np.floor(covpredlstm_6[idx_test,:,0].T),'c',linewidth = 1,alpha= 0.6, linestyle = "dashed", label='LSTM d = 6 - MSE = %f'%mse)
    #mse = np.sum((covdata[idx_test,0,:,0]-covpredlstm_12[idx_test,:,0])*(covdata[idx_test,0,:,0]-covpredlstm_12[idx_test,:,0]))/covdata[idx_test,0,:,0].shape[0]
    #plt.plot(np.floor(covpredlstm_12[idx_test,:,0].T),'c.',linewidth = 1,alpha= 0.6, linestyle = "dashed", label='LSTM d = 12 - MSE = %.2f'%mse)
    mse = np.sum((covdata[idx_test,0,:,0]-covpredlstm_36[idx_test,:,0])*(covdata[idx_test,0,:,0]-covpredlstm_36[idx_test,:,0]))/covdata[idx_test,0,:,0].shape[0]
    plt.plot(np.floor(covpredlstm_36[idx_test,:,0].T),color = 'salmon',marker = 's',linewidth = 1,alpha= 0.6, linestyle = "dashed", label='LSTM d = 36 - MSE = %.2f'%mse)
    mse = np.sum((covdata[idx_test,0,:,0]-covpredlstm_128[idx_test,:,0])*(covdata[idx_test,0,:,0]-covpredlstm_128[idx_test,:,0]))/covdata[idx_test,0,:,0].shape[0]
    plt.plot(np.floor(covpredlstm_128[idx_test,:,0].T),color = 'firebrick',marker = 's',linewidth = 1,alpha= 0.6, linestyle = "dashed", label='LSTM d = 128 - MSE = %.2f'%mse)
    plt.ylabel('Cumulative number of deaths', fontsize=12)
    plt.xlabel('Days', fontsize=12)
    #plt.tick_params(labelright=True)
    plt.grid('on')
    plt.legend()

In [ ]:

preddata_0.shape


In [ ]:
testdata_0 = np.load(data_p + 'test_data_synthetic_d_12_30_particles_cov_idx_0.npy')
preddata_0 = np.load(data_p + 'pred_SMCT_d_12_30_particles_cov_idx_3_var_1.05.npy')
idx_test = 3
plt.figure(figsize=(16,16))
plt.plot(testdata_0[idx_test,0,:-1,0], 'r',linewidth = 2, label='True data')
plt.plot(np.floor(preddata_0[idx_test,:,:-1,0].T),color = 'darkcyan',marker = 'o',linewidth = 2,alpha= 0.6,linestyle = "dashed")
Nsim = 500
for tsp in np.arange(30,44):
    Gauss = np.random.normal(0,1,Nsim)
    idx = np.sum(np.random.multinomial(1, np.ones(30)/30, Nsim)*np.arange(30),axis=1)
    lst_samp = []
    for i in range(Nsim):
        samp = preddata_0[idx_test,idx[i],tsp+1,0] + np.sqrt(1.9)*Gauss[i]
        lst_samp = np.append(lst_samp,samp)
    plt.plot(tsp*np.ones(Nsim),lst_samp,color = 'darkcyan',marker = 'o',alpha= 0.6)
        
plt.ylabel('Cumulative number of deaths', fontsize=12)
plt.xlabel('Days', fontsize=12)
plt.tick_params(labelright=True)
plt.grid('on')
plt.legend()

### Models with noise after training

In [ ]:
covpred_part[idx_test,:,:,0].shape

In [ ]:
np.linspace(35,44,10)

In [ ]:
plt.figure(figsize=(16,16))
for j in range(6):
    plt.subplot(3,2,j+1)
    idx_test = np.random.randint(0,covdata.shape[0])
    plt.plot(covdata[idx_test,0,:,0], 'r',linewidth = 2, label='True data - ' + Admin2[idx_test] + ' ; ' + Province_State[idx_test])
    mse = np.sum((covdata[idx_test,0,:,0]-covpred_part[idx_test,:,:,0].mean(axis=0))*(covdata[idx_test,0,:,0]-covpred_part[idx_test,:,:,0].mean(axis=0)))/covdata[idx_test,0,:,0].shape[0]
    plt.plot(covpred_part[idx_test,:,:,0].mean(axis=0),'m.',linewidth = 1,alpha= 0.3 ,linestyle = "dashed", label='Transf. d = 36 - MSE with mean particle = %.2f'%mse)
    plt.plot(np.linspace(35,44,10),covpred_part[idx_test,:,35::,0].T,'m',linewidth = 1,alpha= 0.3 ,linestyle = "dashed")
    #mse = np.sum((covdata[idx_test,0,:,0]-covpred_12[idx_test,:,:,0])*(covdata[idx_test,0,:,0]-covpred_12[idx_test,:,:,0]))/covdata[idx_test,0,:,0].shape[0]
    #plt.plot(np.floor(covpred_12[idx_test,:,:,0].T),'m.',linewidth = 1,alpha= 0.6,linestyle = "dashed", label='Transf. d = 12 - MSE = %f'%mse)
    #mse = np.sum((covdata[idx_test,0,:,0]-covpred_36[idx_test,:,:,0])*(covdata[idx_test,0,:,0]-covpred_36[idx_test,:,:,0]))/covdata[idx_test,0,:,0].shape[0]
    #plt.plot(np.floor(covpred_36[idx_test,:,:,0].T),'m+',linewidth = 1,alpha= 0.6,linestyle = "dashed", label='Transf. d = 36 - MSE = %f'%mse) 
    #mse = np.sum((covdata[idx_test,0,:,0]-covpred_128[idx_test,:,:,0])*(covdata[idx_test,0,:,0]-covpred_128[idx_test,:,:,0]))/covdata[idx_test,0,:,0].shape[0]
    #plt.plot(np.floor(covpred_128[idx_test,:,:,0].T),'mo',linewidth = 1,alpha= 0.6,linestyle = "dashed", label='Transf. d = 128 - MSE = %f'%mse)
    mse = np.sum((covdata[idx_test,0,:,0]-covpredlstm_6[idx_test,:,0])*(covdata[idx_test,0,:,0]-covpredlstm_6[idx_test,:,0]))/covdata[idx_test,0,:,0].shape[0]
    plt.plot(np.floor(covpredlstm_6[idx_test,:,0].T),'c',linewidth = 1,alpha= 0.6, linestyle = "dashed", label='LSTM d = 6 - MSE = %.2f'%mse)
    mse = np.sum((covdata[idx_test,0,:,0]-covpredlstm_12[idx_test,:,0])*(covdata[idx_test,0,:,0]-covpredlstm_12[idx_test,:,0]))/covdata[idx_test,0,:,0].shape[0]
    plt.plot(np.floor(covpredlstm_12[idx_test,:,0].T),'c.',linewidth = 1,alpha= 0.6, linestyle = "dashed", label='LSTM d = 12 - MSE = %.2f'%mse)
    mse = np.sum((covdata[idx_test,0,:,0]-covpredlstm_36[idx_test,:,0])*(covdata[idx_test,0,:,0]-covpredlstm_36[idx_test,:,0]))/covdata[idx_test,0,:,0].shape[0]
    #plt.plot(np.floor(covpredlstm_36[idx_test,:,0].T),'c+',linewidth = 1,alpha= 0.6, linestyle = "dashed", label='LSTM d = 36 - MSE = %f'%mse)
    mse = np.sum((covdata[idx_test,0,:,0]-covpredlstm_128[idx_test,:,0])*(covdata[idx_test,0,:,0]-covpredlstm_128[idx_test,:,0]))/covdata[idx_test,0,:,0].shape[0]
    plt.plot(np.floor(covpredlstm_128[idx_test,:,0].T),'co',linewidth = 1,alpha= 0.6, linestyle = "dashed", label='LSTM d = 128 - MSE = %.2f'%mse)
    plt.ylabel('Cumulative number of deaths', fontsize=12)
    plt.xlabel('Days', fontsize=12)
    #plt.tick_params(labelright=True)
    plt.grid('on')
    plt.legend()

In [ ]:
covpred_part[idx_test,:,:,0]

In [ ]:
synthetic_data_path = data_p + 'synthetic_dataset_1_feat.npy'

In [ ]:
synthdata = np.load(synthetic_data_path)

In [ ]:
synthdata.shape

In [ ]:
cov_data = np.zeros((3251,46,1))

In [ ]:
for j in range(df.values.shape[0]-10):
    cov_data[j,:,0] = df.values[j+10][50::]

In [ ]:
cov_data.shape

In [ ]:
covdata_data_path = data_p + 'cov_data.npy'
covdata_data_path

In [ ]:
np.save(covdata_data_path,cov_data)

In [ ]:
covid_data_path  = data_p + 'time_series_covid19_deaths_US.csv'

In [ ]:
df = pd.read_csv(covid_data_path)

In [ ]:
df.sort_values(by=[df.columns[11]], inplace=True,ascending = False)

In [ ]:
processed_data = df.drop(df.columns[[0,1,2,3,4,5,6,7,8,9]], axis=1, inplace=True)

In [ ]:
df.drop(df.columns[[0,1]], axis=1, inplace=True)

In [ ]:
df.values[j+10][50::].shape

In [ ]:
for j in range(df.values.shape[0]-10):
    plt.plot(df.values[j+10][50::])
#plt.plot(df.values[1])
#plt.plot(df.values[2])
#plt.plot(df.values[3])
#plt.plot(df.values[4])
#plt.plot(df.values[5])

In [ ]:
df.head()

In [ ]:
testdata.shape

In [ ]:
preddata_100.shape

In [ ]:
testdata[5,0,:,0]

In [ ]:
testdata[13,0,:,0]

In [ ]:
from scipy.stats import norm

In [ ]:
np.random.randint(0,150)

In [ ]:
idx_test = np.random.randint(0,1499)
timestep = np.random.randint(0,23)
true_mean = 0.9*testdata[idx_test,0,timestep-1,0]
xx = np.linspace(true_mean-1,true_mean+1,50)
plt.plot(xx,norm.pdf(xx,true_mean,np.sqrt(0.5)), label='True_pdf')
u = np.zeros(50)
for j in range(20):
    u = u + norm.pdf(xx,preddata[idx_test,j,timestep,0],0.58)
u = u/20
plt.plot(xx,u, label='Predicted_pdf - 20 particles')
u = np.zeros(50)
for j in range(300):
    u = u + norm.pdf(xx,preddata_100[idx_test,j,timestep,0],0.6)
u = u/300
plt.plot(xx,u, label='Predicted_pdf - 100 particles')
plt.legend()

In [ ]:
train_data_synthetic_1_feat_mixt

In [ ]:
preddata[0,:,0,0]

In [ ]:
#plt.plot(testdata[13,0,:,0])
meantrue = []
for j in range(23):
    meantrue =  np.append(meantrue,0.9*testdata[15,0,j,0])
plt.plot(np.cumsum(np.ones(23)),meantrue,'r') 
#plt.plot(preddata[13,0,:,0])
for j in range(19):
    #meantrue =  0.9*testdata[13,0,j,0]
    plt.plot(np.cumsum(np.ones(24)),preddata[15,j,:,0],'b',alpha=0.6)

In [ ]:
idx_test = 8
meantrue = []
for j in range(23):
    meantrue =  np.append(meantrue,0.9*testdata[idx_test,0,j,0])
plt.plot(np.cumsum(np.ones(24)),testdata[idx_test,0,:,0],'r') 
#plt.plot(preddata[13,0,:,0])
for j in range(20):
    #meantrue =  0.9*testdata[13,0,j,0]
    plt.plot(np.cumsum(np.ones(24)),preddata[idx_test,j,:,0],'b',alpha=0.3)

In [ ]:
idx_test = 8
meantrue = []
for j in range(23):
    meantrue =  np.append(meantrue,0.9*testdata[idx_test,0,j,0])
plt.plot(np.cumsum(np.ones(24)),testdata[idx_test,0,:,0],'r') 
#plt.plot(preddata[13,0,:,0])
for j in range(100):
    #meantrue =  0.9*testdata[13,0,j,0]
    plt.plot(np.cumsum(np.ones(24)),preddata_res[idx_test,j,:,0],'b',alpha=0.3)

In [ ]:
preddata_res[idx_test,:,0,0]

In [ ]:
np.sqrt(0.5)

In [ ]:
df = pd.read_csv('imports_prepared.csv')

In [ ]:
df.head()

In [ ]:
from bokeh.charts import output_file, Chord
from bokeh.io import show
from bokeh.sampledata.les_mis import data
 
nodes = data['nodes']
links = data['links']
 
nodes_df = pd.DataFrame(nodes)
links_df = pd.DataFrame(links)
 
source_data = links_df.merge(nodes_df, how='left', left_on='source', right_index=True)
source_data = source_data.merge(nodes_df, how='left', left_on='target', right_index=True)
source_data = source_data[source_data["value"] > 5]
source_data
 
chord_from_df = Chord(source_data, source="name_x", target="name_y", value="value")
output_file('chord-diagram-bokeh.html', mode="inline")
show(chord_from_df)


In [ ]:
fun = lambda u: (np.log(u[0]) + u[0] - 709/15)**2

In [ ]:
bnds = (0, None)

In [ ]:
res = minimize(fun, [.1], method='BFGS')

In [ ]:
res.x[0]/200

In [ ]:
10**300

## Singular Value Decomposition applied to image compression

In [ ]:
# import packages and display image
import numpy as np
import matplotlib.pyplot as plt

from PIL import Image

polar_bears           = Image.open('C:/Users/lecorff/Documents/polar_bears.jpg')

polar_bears_mat       = np.array(list(polar_bears.getdata(band=0)), float)
polar_bears_mat.shape = (polar_bears.size[1], polar_bears.size[0])
polar_bears_mat       = np.matrix(polar_bears_mat)

plt.figure(figsize=(6,4))
plt.imshow(polar_bears_mat, cmap='gray')
plt.title("Polar bears")

In [ ]:
# Perform Singular Value Decomposition
U, sigma, V = np.linalg.svd(polar_bears_mat)
print('Size left singular eigenvectors  ' + str(np.shape(U)))
print('Size right singular eigenvectors  ' + str(np.shape(V)))
print('Size eigenvalues matrix ' + str(np.shape(sigma)))

# Image reconstitution with the largest singular value
reconstimg = np.matrix(U[:, :1]) * np.diag(sigma[:1]) * np.matrix(V[:1, :])
plt.imshow(reconstimg, cmap='gray')
plt.title("Image reconstruction with the largest singular value only")

In [ ]:
nb_singular_values = np.array([5,10,25,100])
k = 0
for i in nb_singular_values:
    reconstimg = np.matrix(U[:, :i]) * np.diag(sigma[:i]) * np.matrix(V[:i, :])
    plt.imshow(reconstimg, cmap='gray')
    title = "Image reconstruction with the " + "n = %s" % i +" largest singular values"
    plt.title(title)
    plt.show()